In [2]:
import os
import re
from multiprocessing import Pool

import pandas as pd


In [3]:
# replace all cells without numbers with NA
def replace_non_digit_string(data_frame):
    mask = ~data_frame[['Salary', 'BirthDate', 'Time']].apply(lambda col: col.astype(str).str.contains(r'\d')).any(axis=1)
    data_frame.loc[mask, ['Name', 'BirthDate', 'Time']] = pd.NA
    return data_frame


In [4]:
# remove all empty strings
def remove_empty_strings(data_frame):
    data_frame = data_frame.dropna(how='all')
    return data_frame


In [5]:
# remove all duplicates
def remove_duplicates(date_frame):
    date_frame = date_frame.drop_duplicates()
    return date_frame


In [6]:
# delete all lines between 1:00:00 and 3:00:00
def remove_by_time(data_frame):
    start_time = pd.to_datetime('01:00:00').time()
    end_time = pd.to_datetime('03:00:00').time()

    data_frame['Time'] = pd.to_datetime(data_frame['Time'], format='%H:%M:%S', errors='coerce').dt.time

    mask = (data_frame['Time'].notna()) & (data_frame['Time'] > start_time) & (data_frame['Time'] < end_time)
    data_frame.loc[mask, ['Name', 'BirthDate', 'Time']] = pd.NA
    return data_frame


In [7]:
df = pd.read_csv('.test_data_set.csv')
# df = pd.read_csv('test.csv')

df.fillna('')

df = replace_non_digit_string(df)   
df = remove_empty_strings(df)
df = remove_duplicates(df)
df = remove_by_time(df)


In [ ]:
# get unique rows for every hour, get median values and mean values
def metrics_calculation(data_frame):
    data_frame['Time'] = pd.to_datetime(data_frame['Time'], format='%H:%M:%S').dt.time
    
    data_frame['Hour'] = pd.to_datetime(data_frame['Time'].astype(str), format='%H:%M:%S').dt.hour
   
    grouped = data_frame.groupby('Hour')

    unique_counts = grouped['Name'].nunique()
    mean_values = grouped['Salary'].mean()
    median_values = grouped['Salary'].median()

    result = pd.DataFrame({
        'UniqueCounts': unique_counts,
        'MeanValues': mean_values,
        'MedianValues': median_values
    })

    return result

metrics = metrics_calculation(df)
metrics


In [23]:
# metrics
# df.head()

merged_data_frame = pd.merge(df, metrics, on='Hour', how='outer')
merged_data_frame


,Name,Salary,BirthDate,Time,Hour,UniqueCounts,MeanValues,MedianValues
0,Worker_0_71,NaN,NaN,00:50:14,0.0,176.0,109524.266667,112634.0
1,Worker_0_151,149789.0,NaN,00:00:53,0.0,176.0,109524.266667,112634.0
2,Worker_0_152,33675.0,1987-08-02,00:37:26,0.0,176.0,109524.266667,112634.0
3,Worker_0_154,42206.0,1987-12-28,00:09:48,0.0,176.0,109524.266667,112634.0
4,Worker_0_157,NaN,NaN,00:24:32,0.0,176.0,109524.266667,112634.0
...,...,...,...,...,...,...,...,...
7841,Worker_9_890,192528.0,1984-03-11,NaT,NaN,NaN,NaN,NaN
7842,Worker_9_891,NaN,2001-12-14,NaT,NaN,NaN,NaN,NaN
7843,Worker_9_894,79244.0,2005-09-25,NaT,NaN,NaN,NaN,NaN
7844,Worker_9_897,134017.0,1973-06-30,NaT,NaN,NaN,NaN,NaN
